In [206]:
import pandas as pd
import re

In [207]:
df = pd.read_csv('csv_data.csv', encoding="utf-8")

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1839 entries, 0 to 1838
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Date Added                                         1839 non-null   object 
 1   category                                           1839 non-null   object 
 2   sellerlink                                         1839 non-null   object 
 3   sellerlink-url                                     1839 non-null   object 
 4   sellerstorefront-url                               1839 non-null   object 
 5   sellerproductcount                                 1513 non-null   object 
 6   sellerratings                                      1411 non-null   object 
 7   sellerdetails                                      1812 non-null   object 
 8   seller business name                               1783 non-null   object 
 9   business

In [209]:
# Describe the dataframe
df.describe()

,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Unnamed: 18,Unnamed: 19,Unnamed: 20
count,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,0.0,0.0,0.0
mean,6.951060,5.808592,7.632409,12.312670,1399.745514,765.173464,NaN,NaN,NaN
std,6.232073,18.117027,19.902394,23.435871,6204.123987,3817.355945,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,5.000000,0.000000,0.000000,0.000000,3.000000,1.000000,NaN,NaN,NaN
75%,15.000000,0.000000,4.000000,14.000000,176.500000,87.500000,NaN,NaN,NaN
max,16.000000,100.000000,100.000000,100.000000,86856.000000,70732.000000,NaN,NaN,NaN


In [210]:
# Display top 5 rows
df.head(5)

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,...,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,15-Nov-20,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,...,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...,NaN,NaN,NaN
1,15-Nov-20,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,...,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...,NaN,NaN,NaN
2,15-Nov-20,Garden,Seller 3,Seller 3-link,Seller 3-storefrontlink,"1-16 of over 2,000 results",85% positive in the last 12 months (75 ratings),67878u6i7 is committed to providing each custo...,Business Name:nantongaohuanfangzhipinyouxiango...,zhongnanshijicheng27zhuang1410haochongchuanqun...,...,0,25,12,164,162,TXYFYP,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN
3,15-Nov-20,Garden,Seller 4,Seller 4-link,Seller 4-storefrontlink,1-16 of 123 results,NaN,7 days home life supermarket is committed to p...,Business Name:Zhengzhou Yaoqiong Trading Company,河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN,...,0,0,0,0,0,SHIFENG,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN
4,15-Nov-20,Garden,Seller 5,Seller 5-link,Seller 5-storefrontlink,"1-16 of over 1,000 results",81% positive in the last 12 months (52 ratings),78 68yui678 is committed to providing each cus...,Business Name:shenzhenfeichangshangmeimaoyiyou...,minzhidadaoshaxialaocun94dongA901longhuaxinqus...,...,0,31,15,224,113,YUOKI99,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN


### <span style="color:red">Extract Product Count</span>

In [211]:
def parse_product_count(_str):
    pattern = '([\d]+(,)*[\d]*)+ results'
    
    # Check if NaN
    if _str != _str:
        return
    
    result = re.search(pattern,_str)
    if result:
        # Extract numerical string
        product_count = result.group(0).split(' ')[0]
        product_count = int(product_count.replace(',',''))
        return product_count

In [212]:
# Create a new column product_count
df['product_count'] = df.apply(lambda x: parse_product_count(x.sellerproductcount),axis=1)

In [213]:
# Print a sample of extracted data
df[['product_count','sellerproductcount']].head()

,product_count,sellerproductcount
0,100000.0,"1-16 of over 100,000 results"
1,NaN,NaN
2,2000.0,"1-16 of over 2,000 results"
3,123.0,1-16 of 123 results
4,1000.0,"1-16 of over 1,000 results"


In [214]:
# Check if any sellerproductcount has not been parsed
df[['product_count','sellerproductcount']].loc[(df['product_count'].isna() & df['sellerproductcount'].notna())]

,product_count,sellerproductcount


In [215]:
# Check if any product_count has been incorrectly extracted
df[['product_count','sellerproductcount']].loc[(df['product_count'].notna() & df['sellerproductcount'].isna())]

,product_count,sellerproductcount


------

### <span style="color:red">Extract No. of Seller ratings and Positive ratings %</span>

In [216]:
def parse_seller_ratings(_str):
    rating_count_pattern = '[\d]+ ratings'
    positive_rating_pattern = '[\d]+%'
    
    # Check if NaN
    if _str != _str:
        return
    
    rating_count = re.search(rating_count_pattern,_str)
    positive_rating = re.search(positive_rating_pattern,_str)
    
    if rating_count:
        rating_count = rating_count.group(0)
        rating_count = int(rating_count.split(' ')[0].replace(',',''))
    if positive_rating:
        positive_rating = int(positive_rating.group(0).replace('%',''))
        
    return (rating_count,positive_rating)

In [217]:
parse_seller_ratings('90% positive in the last 12 months (566 ratings)')

(566, 90)

In [218]:
# Create two new columns rating_count and positive_rating_perc
# df[['rating_count','positive_rating_perc']] = df.apply(lambda x: parse_seller_ratings(x.sellerratings),axis=1)
df[['rating_count','positive_rating_perc']] = df.apply(lambda x: parse_seller_ratings(x.sellerratings), axis=1, result_type='expand')



In [219]:
df.head(5)

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,...,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,Unnamed: 18,Unnamed: 19,Unnamed: 20,product_count,rating_count,positive_rating_perc
0,15-Nov-20,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,...,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...,NaN,NaN,NaN,100000.0,118.0,88.0
1,15-Nov-20,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,...,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...,NaN,NaN,NaN,NaN,566.0,90.0
2,15-Nov-20,Garden,Seller 3,Seller 3-link,Seller 3-storefrontlink,"1-16 of over 2,000 results",85% positive in the last 12 months (75 ratings),67878u6i7 is committed to providing each custo...,Business Name:nantongaohuanfangzhipinyouxiango...,zhongnanshijicheng27zhuang1410haochongchuanqun...,...,164,162,TXYFYP,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN,2000.0,75.0,85.0
3,15-Nov-20,Garden,Seller 4,Seller 4-link,Seller 4-storefrontlink,1-16 of 123 results,NaN,7 days home life supermarket is committed to p...,Business Name:Zhengzhou Yaoqiong Trading Company,河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN,...,0,0,SHIFENG,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN,123.0,NaN,NaN
4,15-Nov-20,Garden,Seller 5,Seller 5-link,Seller 5-storefrontlink,"1-16 of over 1,000 results",81% positive in the last 12 months (52 ratings),78 68yui678 is committed to providing each cus...,Business Name:shenzhenfeichangshangmeimaoyiyou...,minzhidadaoshaxialaocun94dongA901longhuaxinqus...,...,224,113,YUOKI99,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN,1000.0,52.0,81.0


### <span style="color:red"> Parse Seller Details </span>